In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
pet = pd.read_csv("C:/Users/HP/Downloads/pet_adoption_data.csv")
pet.sample(10)

,PetID,PetType,Breed,AgeMonths,Color,Size,WeightKg,Vaccinated,HealthCondition,TimeInShelterDays,AdoptionFee,PreviousOwner,AdoptionLikelihood
488,988,Bird,Parakeet,47,Brown,Small,18.513693,0,0,64,185,0,0
1739,2239,Rabbit,Rabbit,101,Black,Small,3.899572,1,0,42,339,1,0
124,624,Bird,Parakeet,65,Brown,Medium,9.002516,1,0,6,365,0,1
513,1013,Cat,Siamese,94,Orange,Large,19.848611,0,0,33,71,0,0
586,1086,Rabbit,Rabbit,170,White,Medium,7.979766,1,0,35,301,0,1
330,830,Cat,Persian,146,Orange,Small,28.968533,0,0,56,5,1,0
1521,2021,Cat,Persian,173,Gray,Small,3.350727,1,0,88,253,0,0
1681,2181,Rabbit,Rabbit,161,Orange,Large,11.317448,1,0,14,334,0,0
1073,1573,Dog,Poodle,93,Gray,Large,26.039470,1,0,52,451,0,0
1691,2191,Cat,Siamese,68,Orange,Small,17.509160,0,0,15,174,1,0


In [26]:
x = pet[['PetType', 'Breed', 'AgeMonths', 'Color', 'Size', 'WeightKg', 'Vaccinated', 'HealthCondition', 'TimeInShelterDays',
'AdoptionFee', 'PreviousOwner']]
y = pet['AdoptionLikelihood']
x.head()

,PetType,Breed,AgeMonths,Color,Size,WeightKg,Vaccinated,HealthCondition,TimeInShelterDays,AdoptionFee,PreviousOwner
0,Bird,Parakeet,131,Orange,Large,5.039768,1,0,27,140,0
1,Rabbit,Rabbit,73,White,Large,16.086727,0,0,8,235,0
2,Dog,Golden Retriever,136,Orange,Medium,2.076286,0,0,85,385,0
3,Bird,Parakeet,97,White,Small,3.339423,0,0,61,217,1
4,Rabbit,Rabbit,123,Gray,Large,20.498100,0,0,28,14,1


In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
y.shape

(2007,)

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import Pipeline as ImbPipeline
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier



numeric_features = [2, 5, 6, 7, 8, 9, 10]
numeric_transformer = Pipeline(steps=[('num', StandardScaler())])

categorical_features = [0, 1, 3, 4]
categorical_tranformer = Pipeline(steps=[('cat', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_tranformer, categorical_features)])

pipeline = ImbPipeline(steps=[('preprocessor', preprocessor),
                              ('adasyn', ADASYN()),
                              ('Classifier', RandomForestClassifier(class_weight='balanced')),
                             ])


model = pipeline.fit(x_train, (y_train))
print(model)





Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num',
                                                                   StandardScaler())]),
                                                  [2, 5, 6, 7, 8, 9, 10]),
                                                 ('cat',
                                                  Pipeline(steps=[('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [0, 1, 3, 4])])),
                ('adasyn', ADASYN()),
                ('Classifier',
                 RandomForestClassifier(class_weight='balanced'))])


In [29]:
from sklearn.metrics import accuracy_score, classification_report, recall_score
pred = model.predict(x_test)
print('acc: ', accuracy_score(y_test, pred))
print('rep: ', classification_report(y_test, pred))
print('rec: ', recall_score(y_test, pred))

acc:  0.9419568822553898
rep:                precision    recall  f1-score   support

           0       0.95      0.96      0.96       416
           1       0.92      0.89      0.91       187

    accuracy                           0.94       603
   macro avg       0.94      0.93      0.93       603
weighted avg       0.94      0.94      0.94       603

rec:  0.893048128342246


In [30]:
import pickle
with open("pet_model.pkl", 'wb') as f:
    pickle.dump(model, f)

In [35]:
pickle.load(open("pet_model.pkl", 'rb'))
x_new = [['Rabbit', 'Rabbit', '170', 'White', 'Medium', '7.979766', '1', '0', '35', '301', '0']]
pred = model.predict(x_new)[0]
print('Pred: ', pred)

Pred:  1


C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
